In [1]:
# HIDDEN - sets path

import sys
sys.path.insert(1, '../..')

# Work with context items

## Creating the client
Setting the timezone is important as context item timestamps will be given in this timezone.

In [2]:
import keyring
from trendminer_interface import TrendMinerClient

url = "https://cs.trendminer.net"
client_id = "tmInterfaceClient"

client = TrendMinerClient(
    url = url,
    client_id = client_id,
    client_secret=keyring.get_password(url, client_id),
    tz="Europe/Brussels"
)

KeycloakConnectionError: Can't connect to server (HTTPSConnectionPool(host='cs.trendminer.net', port=443): Max retries exceeded with url: /auth/realms/trendminer/protocol/openid-connect/token (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fd035bf7370>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known')))

## Creating context items
We can instantiate an item and post it to the server. For automated workflows, it is typically better to set up a monitor that creates the context items within TrendMiner, and then to update them with your scripts. This avoids the issue that running a script multiple creates duplicate items.

In [ ]:
item = client.context.item(
    context_type = "ANOMALY",
    components=["[CS]BA:CONC.1"],
    events=["2022-01-01", "2022-01-02"],
    keywords=["myexample"],
    fields = {
        "Loss": 12  # key needs to match context field technical name
    }
    
)

item.post()

## Retrieving context items
To retrieve context items we need to instantiate a ContextHub view, with a set of context filters

In [ ]:
chv = client.context.view(
    filters = [
        client.context.filter.interval(("2022-01-01", "2022-01-03")),
        client.context.filter.keywords(["myexample"]),
        client.context.filter.context_types(["ANOMALY"]),
        client.context.filter.components(["[CS]BA:CONC.1"]),
    ]
)

items = chv.search_items()
items

Here are some more filters you might like:

In [ ]:
cf = client.context.filter

# Numeric field
cf.field(field="TMMW", values=[(">", 12), ('<', 15)])

# String/enumeration field
cf.field(field="batch_id", values=['batch1', 'batch2'])

# An empty field
cf.field(field="batch_id", mode='empty')

# Approvals
cf.approval(True)

# User
cf.users(["wdaniels"])

# Duration
cf.duration([('>', '50m'), ('<', 3600)])

# Description (can take wildcard)
cf.description("*monitor*")

# A live period
cf.period('1h')

# The current state
cf.states(['Ended'])

# Closed items only
cf.states(mode='closed');

## Updating items
We can just edit the items and update on the appliance with a put request.

In [ ]:
for item in items:
    item.fields.update({"Loss": 0})
    item.keywords = item.keywords + ["updated"]
    item.put()

## Deleting items
A single item can be deleted by a delete request:

In [ ]:
item.delete()